**Utilizando rede pré-treinada**

Uma rede pré-treinada é uma rede salva que tenha sido anteriormente treinada em um grande dataset, geralmente em uma tarefa de classificação de imagem em larga escala. 

Se esse conjunto de dados original for grande o suficiente e geral o suficiente, as features aprendidas pela rede pré-treinada pode atuar efetivamente como um modelo genérico e podem ser úteis para muitos problemas diferentes de visão computacional

Por exemplo, pode-se treinar uma rede no ImageNet (onde as classes são principalmente animais e objetos do cotidiano) e, em seguida, redirecionar essa rede treinada para algo mais específico. 

Aqui, vamos considerar uma grande convnet treinada do dataset ImageNet (1,4 milhão de imagens rotuladas e 1000 classes diferentes). O ImageNet contém muitas classes de animais, incluindo diferentes espécies de cães e gatos (nosso objetivo)

Usaremos a arquitetura VGG16, desenvolvida por Karen Simonyan e Andrew Zisserman em 2014, uma arquitetura de convnet simples e amplamente usada para o ImageNet. 

Há duas maneiras de aproveitar uma rede pré-treinada: extração de features e fine tuning. 

**Extração de features**

Utiliza as representações aprendidas por uma rede anterior para extrair features de novas amostras. Estas features são executadas através de um novo classificador, treinado a partir do zero.

Lembre-se que as convnet usadas para classificação são divididas em duas partes: camadas convolucionais e de pooling para extração de features (chamada de base convolucional) e camadas densas para classificação. 

Normalmente, as features aprendidas pelas camadas convolucionais são mais genéricas e reutilizáveis (as representações aprendidas pelas camadas densass de classificação são mais específicas e não devem ser reutilizadas).

Vamos usar a base convolucional da rede VGG16, treinada no ImageNet, para extrair features de imagens de gatos e cães e, em seguida, treinar um classificador de gato versus cachorro sobre essas features.

O modelo VGG16, entre outros, vem pré-embalado com Keras. 

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

Argumentos:

**weights**, de onde queremos inicializar o modelo

**include_top**, se refere à inclusão ou não do classificador densely-connected na parte superior da rede. Por padrão, esse classificador densely-connected corresponderia às 1000 classes do ImageNet. Como pretendemos usar nosso próprio classificador densely-connected (com apenas duas classes, gato e cachorro), não vamos incluí-lo.

**input_shape**, o formato dos tensores de imagem (opcional). 

In [ ]:
conv_base.summary()

O feature-map final tem forma (4, 4, 512). Esse é a fature onde colocaremos um classificador fully-connecteds.

Aqui, podemos proceder de duas formas:

1 - Executando a base convolucional sobre nosso conjunto de dados, gravando sua saída em um array Numpy no disco e, em seguida, usando esses dados como entrada para um classificador (fully-connecteds). 

Solução rápida porque requer a execução da base convolucional apenas uma vez para cada imagem de entrada. 

No entanto, essa técnica não nos permitiria aproveitar o data augmentation.

2 - Estendendo o modelo que temos (conv_base) adicionando camadas Densas na parte superior e executando tudo de ponta a ponta nos dados de entrada.

Vamos tentar a primeira opção:

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O cats_and_dogs_filtered.zip

In [ ]:
import os
import zipfile

local_zip = 'cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = 'data/cats_and_dogs_filtered/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
# test_dir = os.path.join(base_dir, 'test')

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
# test_features, test_labels = extract_features(test_dir, 1000)

As features extraídos são da forma (samples, 4, 4, 512). Vamos adicionar um classificador densamente conectado; portanto, primeiro devemos aplicar flatten para (samples, 8192):

In [ ]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
# test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Observe que a acurácia do modelo no conjunto de validação já foii melhor do que anteriormente (com data augmentation e sem usar a VGG16)

*Obs.: Veja o notebook "unidade03_keras_extract-features.ipynb" antes de continuarmos*

A outra forma de fazer é (opção 2) estendendo o modelo que temos (conv_base) adicionando camadas Densas na parte superior e executando tudo de ponta a ponta nos dados de entrada. Muito mais lento (necessário o uso de GPU).

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
input_shape=(None, 150, 150, 3)
model.build(input_shape)
model.summary()

A base convolucional do VGG16 possui 14.714.688 parâmetros, o que é muito grande. O classificador que estamos adicionando no topo tem 2 milhões de parâmetros.

Antes de compilar e treinar nosso modelo, uma coisa muito importante a fazer é "freeze" a base convolucional (VGG16). "Freezing" uma camada ou conjunto de camadas significa impedir que seus pesos sejam atualizados durante o treinamento. 

Se não fizer isso, as representações aprendidas anteriormente pela base convolucional (VGG16) serão modificadas durante o treinamento. Como as camadas densas na parte superior são inicializadas aleatoriamente, atualizações dos pesos seriam propagadas pela rede, destruindo o modelo pré-treinado.


In [ ]:
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))

In [ ]:
# Freezing a base convolucional
conv_base.trainable = False

In [ ]:
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

Com essa configuração, apenas os pesos das duas camadas Densas que adicionamos serão treinados

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=2e-5),
              metrics=['acc'])

history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

In [ ]:
model.save('cats_and_dogs_small_3.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

**Fine-tuning**

Consiste em descongelar algumas das camadas superiores de uma base de modelo congelada usada para extração de features e treinar em conjunto a parte recém-adicionada do modelo (no nosso caso, o classificador) e essas camadas superiores. 

Isso ajusta levemente as representações mais abstratas do modelo que está sendo reutilizado, a fim de torná-las mais relevantes para o problema em questão.

Congelamos a base convolucional da VGG16 para poder treinar um classificador inicializado aleatoriamente no topo. 

Pelo mesmo motivo, só é possível ajustar as camadas superiores da base convolucional depois que o classificador na parte superior já tiver sido treinado. 

Portanto, as etapas para o fine-tuning de uma rede são as seguintes:

1) Adicione sua rede personalizada (a nossa rede) em cima de uma rede básica já treinada (VGG16).
2) Congele a rede base (VGG16).
3) Treine a parte que adicionada (o que acabamos de fazer).
4) Descongele algumas camadas na rede base (da VGG16).
5) Treine em conjunto essas duas camadas (VGG16 com algumas camadas descongeladas mais o nosso modelo de classificação).

Vamos descongelar nossa conv_base e congelaremos camadas individuais dentro dela.

Ajustaremos as três últimas camadas convolucionais, o que significa que todas as camadas até block4_pool (veja a VGG original abaixo) devem ser congeladas e as camadas block5_conv1, block5_conv2 e block5_conv3 devem ser treináveis.

Optamos por apenas ajudar as últimas camadas e não as primeiras porque estas últimas aprendem representação mais específicas (para o nosso caso, preferimos as representações mais genéricas)

In [ ]:
# VGG16 original (base convolucional)
conv_base.summary()

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-5),
              metrics=['acc'])

history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      # epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
model.save('cats_and_dogs_small_4.h5')

In [ ]:
def smooth_curve(points, factor=0.8):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

plt.plot(epochs,
         smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,
         smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,
         smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,
         smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Se você tiver um conjunto de teste, utile-o para testar o modelo  

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

Referência: François Chollet. Deep Learning with Python. November 2017